In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:86% !important;}
div.cell.code_cell.rendered{width:100%;}
div.CodeMirror {font-family:Consolas; font-size:12pt;}
div.output {font-size:15pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:12pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:15px;}
</style>
"""))

[ RAG 구현 절차 ]

```
1.	문서의 내용을 읽는다(document_loader를 이용)
(1)	https://python.langchain.com/v0.2/docs/integrations/document_loaders/ 
(2)	https://python.langchain.com/v0.2/docs/integrations/document_loaders/microsoft_word/
%pip install --upgrade --quiet  docx2txt
2.	문서를 쪼갠다(한번에 이해하고 처리할 수 있는 입력+출력 토큰수가 제한)
(1)	 https://python.langchain.com/v0.2/docs/how_to/recursive_text_splitter/#splitting-text-from-languages-without-word-boundaries 
%pip install -qU langchain-text-splitters
3.	쪼갠 문서를 임베딩하여 vector database에 넣음
(1)	OpenAIEmbeddings나 UpstageEmbeddings이용해서 임베딩
(2)	https://python.langchain.com/v0.2/docs/integrations/vectorstores/chroma/  
%pip install –q langchain-chroma
4.	질문을 이용해 유사도 검색
5.	유사도 검색한 문서를 LLM에 질문으로 전달하여 답변 얻음(제공되는 Prompt활용)
(1)	https://python.langchain.com/v0.2/docs/tutorials/rag/
%pip install –q langchain langchainhub
http://smith.langchain.com에서 key생성 .env key(LANGCHAIN_API_KEY) 추가

```

# 2. 문서를 쪼개면서 읽기(o)

In [7]:
# %pip install langchain_upstage

   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   ---------------------------------------  2.4/2.4 MB 12.2 MB/s eta 0:00:01
   ---------------------------------------- 2.4/2.4 MB 11.3 MB/s eta 0:00:00

   ---------------------------------------- 0/3 [pypdf]
   ---------------------------------------- 0/3 [pypdf]
  Attempting uninstall: tokenizers
   ---------------------------------------- 0/3 [pypdf]
    Found existing installation: tokenizers 0.21.4
   ---------------------------------------- 0/3 [pypdf]
    Uninstalling tokenizers-0.21.4:
   ---------------------------------------- 0/3 [pypdf]
      Successfully uninstalled tokenizers-0.21.4
   ---------------------------------------- 0/3 [pypdf]
   ------------- -------------------------- 1/3 [tokenizers]
   ---------------------------------------- 3/3 [langchain_upstage]

Note: you may need to restart the kernel to use updated packages.


In [2]:
import time
start = time.time()
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
loader = Docx2txtLoader('./tax_docs/소득세법(법률)(제20615호)(20250701).docx')
text_splitter = RecursiveCharacterTextSplitter(  # 문서를 쪼개는 기준이 문자수
    chunk_size=1500, #문서를 쪼갤때 1500글자씩 쪼개
    chunk_overlap=200
)
# 1번째 chunk 1~1450글자
# 2번째 chunk 1250~1750글자
documents = loader.load_and_split(text_splitter=text_splitter)
runtime = time.time() - start
print('문서 쪼개면서 읽는 시간 :', runtime)

문서 쪼개면서 읽는 시간 : 3.705723285675049


# 3. 쪼갠문서를 임베딩 -> 벡터 데이터베이스 저장
- 임베딩 모델 : upstage의 text-embedding-3-large (기본:text-embedding-ada-002)
- 벡터 데이터베이스 : chroma

In [3]:
# https://python.langchain.com/v0.2/docs/integrations/text_embedding/upstage
from dotenv import load_dotenv
from langchain_upstage import UpstageEmbeddings
load_dotenv()
embeddings = UpstageEmbeddings(
    model="solar-embedding-1-large"
    # model="embedding-query"
)

In [4]:
doc_result = embeddings.embed_documents(
    ["소득세법 어쩌구 저쩌구", documents[0].page_content]
)
print(len(doc_result), len(doc_result[0]), len(doc_result[1]))

2 4096 4096


In [7]:
%%time
from langchain_chroma import Chroma
# 데이터를 처음 저장할 때
# database = Chroma.from_documents(                                 
#     documents=documents,
#     embedding=embeddings,
#     collection_name="tax-collection", # 생략시 이름 랜덤
#     persist_directory='./chroma_upstage'      # 생략시 로컬데이터베이스에 저장안됨. 프로그램 종료시 db날라감
# )
# 이미 저장된 vector DB를 사용할 때
database = Chroma(
    embedding_function=embeddings,
    collection_name="tax-collection",
    persist_directory='./chroma_upstage'
)

CPU times: total: 0 ns
Wall time: 29.1 ms


# 4. vector DB에 질문과 유사도 검색(답변 생성을 위한 retrieval)

In [8]:
query = "연봉 5천만원인 직장인의 소득세는 얼마인가요?"
retrieved_docs = database.similarity_search(query,
                                           k=3) # 기본 k는 4

In [19]:
retrieved_docs

[Document(id='d2f774e3-1ad2-4bf1-8697-cb93e02bad6d', metadata={'source': './tax_docs/소득세법(법률)(제20615호)(20250701).docx'}, page_content='[전문개정 2009. 12. 31.]\n\n\n\n제10조(납세지의 변경신고) 거주자나 비거주자는 제6조부터 제9조까지의 규정에 따른 납세지가 변경된 경우 변경된 날부터 15일 이내에 대통령령으로 정하는 바에 따라 그 변경 후의 납세지 관할 세무서장에게 신고하여야 한다.\n\n[전문개정 2009. 12. 31.]\n\n\n\n제11조(과세 관할) 소득세는 제6조부터 제10조까지의 규정에 따른 납세지를 관할하는 세무서장 또는 지방국세청장이 과세한다.\n\n[전문개정 2009. 12. 31.]\n\n\n\n제2장 거주자의 종합소득 및 퇴직소득에 대한 납세의무 <개정 2009. 12. 31.>\n\n\n\n제1절 비과세 <개정 2009. 12. 31.>\n\n\n\n제12조(비과세소득) 다음 각 호의 소득에 대해서는 소득세를 과세하지 아니한다. <개정 2010. 12. 27., 2011. 7. 25., 2011. 9. 15., 2012. 2. 1., 2013. 1. 1., 2013. 3. 22., 2014. 1. 1., 2014. 3. 18., 2014. 12. 23., 2015. 12. 15., 2016. 12. 20., 2018. 3. 20., 2018. 12. 31., 2019. 12. 10., 2019. 12. 31., 2020. 6. 9., 2020. 12. 29., 2022. 8. 12., 2022. 12. 31., 2023. 8. 8., 2023. 12. 31., 2024. 12. 31.>\n\n1. 「공익신탁법」에 따른 공익신탁의 이익\n\n2. 사업소득 중 다음 각 목의 어느 하나에 해당하는 소득\n\n가. 논ㆍ밭을 작물 생산에 이용하게 함으로써 발생하는 소득\n\n나. 1개의 주택을 소유하는 자의 주택임대소득(

# 5. 유사도 검색으로 가져온 문서를 질문과 같이 LLM 전달하여 답변 생성

In [9]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4.1-nano")

In [10]:
prompt = f"""[identity]
- 당신은 최고의 한국 소득세 전문자입니다
- [context]를 참고해서 사용자의 질문에 답변해 주세요
[context]는 다음과 같아요
{retrieved_docs}
Question : {query}"""

In [11]:
ai_message = llm.invoke(prompt)

In [12]:
print(ai_message.content)

연봉 5천만원인 직장인의 소득세를 계산하기 위해서는 여러 공제와 세율을 고려해야 합니다. 아래는 일반적인 계산 과정입니다.

1. 총급여액 : 50,000,000원

2. 근로소득공제 계산
- 최대 공제액 : 2,000만원
- 5천만원은 공제액 2,000만원으로 제한
- 근로소득공제액 : 20,000,000원

3. 과세표준(과세대상 소득) 산출
- 총급여액(50,000,000원) - 근로소득공제(20,000,000원) = 30,000,000원

4. 기본 세율 적용
- 1,200만원 이하의 부분에 대한 세율은 6%
- 1,200만원 초과 4,600만원 이하 구간은 15%

하지만, 구간별 세율을 적용하기 위해서는 누진세 계산이 필요하므로 대략적인 세액을 계산해 보겠습니다.

[한국 소득세 구간별 세율]

| 과세표준 구간 | 세율 | 누진공제액 |
|--------------|--------|------------|
| 1,200만원 이하 | 6%  | 0원       |
| 1,200만원 초과 ~ 4,600만원 이하 | 15% | 108만원 |
| 4,600만원 초과 ~ 8,800만원 이하 | 24% | 522만원 |
| 8,800만원 초과 ~ 1억 5천만원 이하 | 35% | 1,490만원 |
| 1억 5천만원 초과 | 38% | 1,950만원 |

그럼, 과세표준 30,000,000원에 대한 세액은:

- 1,200만원까지: 1,200만원 × 6% = 72,000원
- 나머지 30,000,000 - 12,000,000 = 18,000,000원에 대해
  - 15% 적용 후, 누진공제 108만원을 공제하게 됩니다.

하지만, 좀 더 직관적으로 계산해서:

- 1,200만원까지: 72,000원
- 1,200만원 초과 4,600만원까지(3,400만원): 3,400만원 × 15% = 510만원
- 4,600만원 초과 30,000만원까지(25,400만원): 25,400만원 × 24% = 6,096만원

총 세금 = 72,000 + 510만원 + 6,096만원 =

# 5. Augmentation을 위한 제공되는 Prompt활용하여 langchain으로 답변 생성

In [13]:
query = "연봉 5천만원인 직장인의 소득세는 얼마인가요?"

from langchain import hub
prompt = hub.pull("rlm/rag-prompt")
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

### RetrievalQA를 통해 LLM전달 (create_retrieval_chain이 대체)
```
 query -> retriever전달(백터 검색 수행) 
 -> retrieval문서 -> prompt의 {context}에 삽입
 -> query -> prompt의 {question}에 삽입
```

In [14]:
from langchain.chains import RetrievalQA
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever = database.as_retriever(search_kwargs={'k':5}),
    chain_type_kwargs={"prompt":prompt}
)

In [15]:
ai_message = qa_chain.invoke({"query":query})

In [16]:
ai_message

{'query': '연봉 5천만원인 직장인의 소득세는 얼마인가요?',
 'result': '연봉 5천만원인 직장인의 소득세는 약 1,041,000원입니다. 이는 근로소득공제 및 기타 공제, 세율 구간에 따라 계산된 추정치입니다. 정확한 금액은 개인의 상세 상황에 따라 달라질 수 있습니다.'}